# 1,2. 데이터 읽기

In [25]:
import pandas as pd
import numpy as np

In [26]:
u_cols = ['user_id','age','sex','occupation','zip_code']
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']


users = pd.read_csv('ml-100k/u.user',sep='|',names=u_cols)
movies = pd.read_csv('ml-100k/u.item',sep='|',names=i_cols,encoding='latin-1')
ratings = pd.read_csv('ml-100k/u.data',sep='\t',names=r_cols)

ratings = ratings.drop('timestamp',axis=1)
movies = movies[['movie_id','title']]

In [27]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

def score(model):
    id_pairs = zip(x_test['user_id'],x_test['movie_id'])
    y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_pred, y_true)
def score_knn(model,neighbor_size=0):
    id_pairs = zip(x_test['user_id'],x_test['movie_id'])
    y_pred = np.array([model(user,movie,neighbor_size) for (user,movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_pred, y_true)

# 3. 기본 CF 알고리즘

```
유저가 각 영화별로 남긴 평점을 벡터로 보고, 각 유저 벡터간의 cosine similarity를 계산

ex)
users = [[0,1,0],
         [1,0,0],
         [0,2,0]]
         
cosine_similarity(users,users)

output
array([[1., 0., 1.],
       [0., 1., 0.],
       [1., 0., 1.]])
       
```

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

x = ratings.copy()
y = ratings.user_id

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.25,stratify=y)
rating_matrix = x_train.pivot(index='user_id',columns='movie_id',values='rating')

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.182154,0.055385,0.050865,0.253551,0.303828,0.320048,0.268179,0.083701,0.281049,...,0.263514,0.111053,0.209291,0.195930,0.126533,0.095288,0.213782,0.168002,0.161409,0.321843
2,0.182154,1.000000,0.064378,0.057554,0.052443,0.207537,0.085640,0.109787,0.132513,0.117765,...,0.112991,0.206312,0.232086,0.375504,0.196906,0.165287,0.149097,0.140297,0.094402,0.086339
3,0.055385,0.064378,1.000000,0.339246,0.029723,0.088219,0.057133,0.064002,0.084117,0.079557,...,0.019063,0.039736,0.125469,0.066856,0.113579,0.016059,0.153743,0.090568,0.103045,0.036136
4,0.050865,0.057554,0.339246,1.000000,0.000000,0.057727,0.031987,0.127406,0.061672,0.039727,...,0.014675,0.000000,0.114938,0.123522,0.132241,0.041210,0.162843,0.079682,0.151332,0.037092
5,0.253551,0.052443,0.029723,0.000000,1.000000,0.164763,0.287694,0.173458,0.063220,0.148189,...,0.240069,0.039197,0.068138,0.050649,0.091860,0.055973,0.223721,0.166767,0.096747,0.198260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.095288,0.165287,0.016059,0.041210,0.055973,0.092469,0.087528,0.096476,0.053371,0.027504,...,0.046567,0.213547,0.202291,0.184338,0.354170,1.000000,0.069737,0.209171,0.018025,0.105469
940,0.213782,0.149097,0.153743,0.162843,0.223721,0.256667,0.222534,0.159454,0.088709,0.288203,...,0.263515,0.070458,0.112482,0.119223,0.094224,0.069737,1.000000,0.058056,0.188165,0.249575
941,0.168002,0.140297,0.090568,0.079682,0.166767,0.104802,0.037383,0.086609,0.141036,0.078786,...,0.049113,0.136498,0.236354,0.177602,0.279514,0.209171,0.058056,1.000000,0.065349,0.081277


In [29]:
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.dropna()
        sim_scores = sim_scores.drop(none_rating_idx)
        mean_rating = np.dot(sim_scores,movie_ratings)/sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

score(CF_simple)

1.0188095710698413

# 4. 이웃을 고려한 CF

In [30]:
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.drop(none_rating_idx)
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:          
            # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
##### (3) Neighbor size가 지정된 경우
        else:                       
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1: 
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도를 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                # 영화 rating을 neighbor size만큼 받기
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 최종 예측값 계산 
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

score_knn(cf_knn, neighbor_size=30)

1.0099960396408152

In [31]:
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [32]:
def recom_movie(user_id, n_items, neighbor_size=30):
    # 현 사용자가 평가한 영화 가져오기
    user_movie = rating_matrix.loc[user_id].copy()
    for movie in rating_matrix:
        # 현 사용자가 이미 평가한 영화는 제외 (평점을 0으로)        
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie] = 0
        # 현 사용자가 평가하지 않은 영화의 예상 평점 계산
        else:
            user_movie.loc[movie] = cf_knn(user_id, movie, neighbor_size)
    # 영화를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌
    movie_sort = user_movie.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

recom_movie(user_id=2, n_items=5, neighbor_size=30)

movie_id
1293                     Ayn Rand: A Sense of Life (1997)
1189                              That Old Feeling (1997)
1467                                     Cure, The (1995)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

# 5. 최적의 이웃 크기 결정

In [34]:
# train set으로 full matrix와 cosine similarity 구하기 
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score_knn(cf_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 0.8073
Neighbor size = 20 : RMSE = 0.8748
Neighbor size = 30 : RMSE = 0.9021
Neighbor size = 40 : RMSE = 0.9174
Neighbor size = 50 : RMSE = 0.9261
Neighbor size = 60 : RMSE = 0.9328


# 6. 사용자의 평가경향을 고려한 CF

In [36]:
rating_mean = rating_matrix.mean(axis=1)
rating_bias = (rating_matrix.T - rating_mean).T

def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id].copy()
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
##### (3) Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산            
            if len(sim_scores) > 1:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

score_knn(CF_knn_bias, 30)

0.8391740713178293

# 7. 그 외의 CF 정확도 개선 방법

In [37]:
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0)

In [38]:
def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id]
        # 현 movie에 대한 rating이 없는 사용자 표시
        no_rating = movie_ratings.isnull()
        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        common_counts = counts[user_id]
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 표시
        low_significance = common_counts < SIG_LEVEL
        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
##### (3) Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산            
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

SIG_LEVEL = 3
MIN_RATINGS = 2
score_knn(CF_knn_bias_sig, 30)

0.840036552411263

# 8. 사용자 기반 CF와 아이템 기반 CF

In [41]:
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

In [42]:
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:      # 현재 영화가 train set에 있는지 확인
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_IBCF)

0.9764117851881334